In [1]:
import numpy as np
import polars as pl
import gsd.hoomd
import schmeud._schmeud as schmeud_rs
from schmeud._schmeud import statics
from schmeud import ml
from tqdm import tqdm

import glob
import os
import pathlib
import pickle
import signac
import freud
from numba import njit

from dataclasses import dataclass
from collections import defaultdict

import matplotlib.pyplot as plt
from scipy import stats
import hoomd

In [2]:
from monk import workflow, utils, prep as prep_mod

In [3]:
parent = pathlib.Path(os.getcwd()).parent / "config.yaml"
config = workflow.get_config(parent.as_posix())
parent, config

(PosixPath('/home/ian/Projects/work/monk/workflows/2d-osc-shear/config.yaml'),
 {'root': '/media/ian/Data2/monk/2d-osc-shear',
  'origin': '/media/ian/Data2/monk/2d-esl'})

In [4]:
project: signac.Project = signac.get_project(root=config['root'])
project.doc

{'avail_seed': 18, 'dt': 0.005, 'step_unit': 200, 'equil_time': 100, 'min_periods': 20, 'dumps': 40, 'period_times': [30.0, 100.0, 300.0, 1000.0], 'max_shears': [0.01, 0.02, 0.03, 0.05, 0.08, 0.12, 0.04, 0.06, 0.07], '_status': {}}

In [5]:
pipe = None
pipe0 = None
pipe1 = None
with open("svc.pkl", "rb") as f:
    pipe = pickle.load(f)

with open("svc_type0.pkl", "rb") as f:
    pipe0 = pickle.load(f)

with open("svc_type1.pkl", "rb") as f:
    pipe1 = pickle.load(f)

In [6]:
@dataclass(frozen=True, eq=True)
class Statepoint:
    max_shear: float
    period: float
    temp: float
    prep: str

In [9]:

for job in project.find_jobs({"prep": "HTL"}):
    print(job)
    prep = job.sp["prep"]
    # if prep != "ESL":
    #     continue
    # print(prep)
    
    experiments = sorted(glob.glob(job.fn("longer_experiments/*/*/traj-fire_period-*.gsd")))
    if len(experiments) == 0:
        continue
    # inter_computer = InteractionComputer(experiments[0], job.sp["pot"], frame=0)
    for exper in experiments:
        max_shear = utils.extract_between(exper, "max-shear-", "/")
        period = utils.extract_between(exper, "period-", ".gsd")
        temp = utils.extract_between(exper, "temp-", "/")
        df_path = f"longer_experiments/max-shear-{max_shear}/temp-{temp}/auto-encoder-dataset-yieldstress_period-{period}.parquet"
        sp = Statepoint(max_shear=float(max_shear), period=float(period), temp=float(temp), prep=prep)
        
        if float(period) != 1000.0 or float(temp) != 0.019836 or float(max_shear) != 0.04:
            continue

        # if job.isfile(df_path):
        #     # dataset = pl.read_parquet(job.fn(df_path))
        #     # output[sp].append(dataset)
        #     continue

        traj = gsd.hoomd.open(exper)

        oframes = []
        ostrain = []
        oid = []
        osoft = []
        osfs = []
        # od2min_irr_10 = []
        # od2min_rev_10 = []
        # od2min_irr_20 = []
        # od2min_rev_20 = []

        frame = len(traj) - 1

        # start = 1
        # end = 101
        # cycle_start_idx = lambda i: -1 + i*40
        # # for frame in tqdm(range(cycle_start_idx(start), cycle_start_idx(end),
        # # 40)):
        # for frame in range(cycle_start_idx(start), cycle_start_idx(end), 40*20):

        # inter_computer.set_frame(frame)
        # virials = inter_computer.get_virials()
        # xx = virials[:, 0]
        # yy = virials[:, 3]
        # xy = virials[:, 1]

        # inter_computer.set_frame(frame + 1)
        # virials = inter_computer.get_virials()
        # xx1 = virials[:, 0]
        # yy1 = virials[:, 3]
        # xy1 = virials[:, 1]

        # inter_computer.set_frame(frame + 2)
        # virials = inter_computer.get_virials()
        # xx2 = virials[:, 0]
        # yy2 = virials[:, 3]
        # xy2 = virials[:, 1]

        # xxs = np.stack((xx, xx1, xx2), axis=-1)
        # yys = np.stack((yy, yy1, yy2), axis=-1)
        # xys = np.stack((xy, xy1, xy2), axis=-1)

        snap = traj[frame]

        typeids = snap.particles.typeid
        softb = np.zeros_like(typeids, dtype=np.float32)

        query_indices0 = np.arange(snap.particles.N)[typeids == 0]
        sfs0 = ml.compute_structure_functions_snap(snap, query_indices0)
        soft0 = pipe0.decision_function(sfs0)

        query_indices1 = np.arange(snap.particles.N)[typeids == 1]
        sfs1 = ml.compute_structure_functions_snap(snap, query_indices1)
        soft1 = pipe1.decision_function(sfs1)

        softb[typeids == 0] = soft0
        softb[typeids == 1] = soft1

        # softness.append(softb)

        all_sfs = np.zeros((snap.particles.N, sfs1.shape[1]), dtype=np.float32)

        all_sfs[typeids == 0] = sfs0
        all_sfs[typeids == 1] = sfs1

        # get softness within r_max
        # nlist_query = freud.locality.LinkCell.from_system(snap)
        # nlist = nlist_query.query(snap.particles.position, {'exclude_ii': True, "r_max": 40.0}).toNeighborList()

        # neighbor_softs = softb[nlist.point_indices]
        # local_soft = get_local_softness(softb, nlist.segments, nlist.neighbor_counts, nlist.point_indices, nlist.distances, 40.0, 40)
        

        frames = np.ones_like(typeids, dtype=np.int32) * frame

        strains = np.ones_like(typeids, dtype=np.float32) * float(max_shear)

        # snap_high = traj[10 + frame]
        # snap_later = traj[40 + frame]
        # snap_low = traj[30 + frame]

        # box = snap.configuration.box[:]
        # box_high = snap_high.configuration.box[:]
        # box_later = snap_later.configuration.box[:]
        # box_low = snap_low.configuration.box[:]

        # nlist_query = freud.locality.LinkCell.from_system(snap)
        # nlist = nlist_query.query(snap.particles.position, {'num_neighbors': 10}).toNeighborList()

        # d2min_irr = schmeud_rs.dynamics.d2min_frame(snap.particles.position[:, :2], snap_later.particles.position[:, :2], nlist.query_point_indices, nlist.point_indices, (box, box_later))
        # d2min_rev = schmeud_rs.dynamics.d2min_frame(snap_high.particles.position[:, :2], snap_low.particles.position[:, :2], nlist.query_point_indices, nlist.point_indices, (box_high, box_low))
        # # d2min_c = np.sqrt(np.square(d2min_rev) + np.square(d2min_irr)) * np.sign(d2min_rev - d2min_irr)

        # nlist_query = freud.locality.LinkCell.from_system(snap)
        # nlist = nlist_query.query(snap.particles.position, {'num_neighbors': 20}).toNeighborList()

        # d2min_irr_20 = schmeud_rs.dynamics.d2min_frame(snap.particles.position[:, :2], snap_later.particles.position[:, :2], nlist.query_point_indices, nlist.point_indices, (box, box_later))
        # d2min_rev_20 = schmeud_rs.dynamics.d2min_frame(snap_high.particles.position[:, :2], snap_low.particles.position[:, :2], nlist.query_point_indices, nlist.point_indices, (box_high, box_low))
        # d2min_c = np.sqrt(np.square(d2min_rev) + np.square(d2min_irr)) *
        # np.sign(d2min_rev - d2min_irr)
        
        oframes.append(frames)
        ostrain.append(strains)
        oid.append(typeids)
        osoft.append(softb)
        osfs.append(all_sfs)
        # od2min_irr_10.append(d2min_irr)
        # od2min_rev_10.append(d2min_rev)
        # od2min_irr_20.append(d2min_irr_20)
        # od2min_rev_20.append(d2min_rev_20)

        frames = np.concatenate(oframes)
        strains = np.concatenate(ostrain)
        typeids = np.concatenate(oid)
        softb = np.concatenate(osoft)
        all_sfs = np.concatenate(osfs)
        # d2min_irr = np.concatenate(od2min_irr_10)
        # d2min_rev = np.concatenate(od2min_rev_10)
        # d2min_irr_20 = np.concatenate(od2min_irr_20)
        # d2min_rev_20 = np.concatenate(od2min_rev_20)

        dataset = pl.DataFrame({"frame": frames, "strain": strains, "id": np.array(typeids), "soft": np.array(softb), "sfs": all_sfs})
    # dataset = dataset.explode(["id", "soft"]).reset_index(drop=True)
        dataset.write_parquet(job.fn(df_path), use_pyarrow=True)
    #     break
    # break


6976a7c10ada4159010403f8027f3ea5


ecce68c50e28a33684826f28780bf6e9
cfa1e4c0007fd65d020f809715a6f0b4
1083a2a09eec2c6cf215ebc7a9706cac
634e14bc75836df75158fb15f379d8bb
9158e73b2e4c9fd1a565e1e5d1bc3af5
